# PyTorchでニューラルネットワーク基礎 #03 【分類問題】

* Qiitaの記事と連動しています
* 「あっさり、こってり、塩」ラーメンの分類問題
* ramen_data.csv

In [1]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split

## 精度，正答率の関数

予測値と教師データで等しい値なら正解として、正解数/問題数で精度を求める単純な精度


In [2]:
def accuracy(y, t):
    _,argmax_list = torch.max(y, dim=1)
    accuracy = sum(argmax_list == t).item()/len(t)
    return accuracy

## データの読み込み
* numpy.loadtxtを利用する。
* ラーメンタイプはID化されたラーメンタイプIDの列を利用する

In [3]:
filename = "./ramen_data.csv"
x = np.loadtxt(filename, delimiter=",", skiprows=1, usecols=(0,1,2,3,4))
t = np.loadtxt(filename, delimiter=",", skiprows=1, usecols=(6))
# numpyからtorchテンソルへ
x = torch.FloatTensor(x)   # 入力するデータは Float
t = torch.LongTensor(t)    # ラベル側はLong

# データの分割 
x, x_test, t, t_test = train_test_split(x,t, test_size=0.1, random_state=55)

In [4]:
x.shape, x_test.shape, t.shape, t_test.shape

(torch.Size([450, 5]),
 torch.Size([50, 5]),
 torch.Size([450]),
 torch.Size([50]))

In [5]:
# モデルの定義
class DNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(in_features=5, out_features=10)
        self.act = nn.ReLU()
        self.fc2 = nn.Linear(in_features=10, out_features=3) 
    
    def forward(self, x):
        h = self.fc1(x)
        h = self.act(h)
        y = self.fc2(h)
        return y

model = DNN()

In [6]:
model

DNN(
  (fc1): Linear(in_features=5, out_features=10, bias=True)
  (act): ReLU()
  (fc2): Linear(in_features=10, out_features=3, bias=True)
)

In [7]:
criterion = nn.CrossEntropyLoss()                          # 損失関数：cross_entropy
optimizer = torch.optim.Adam(model.parameters(), lr=0.05) # 学習率：lr=0.001がデフォルト

In [8]:
# 機械学習
LOOP = 50

for epoch in range(LOOP):
    optimizer.zero_grad()                 # 勾配の初期化
    y = model(x)                          # 順伝播・NNの計算
    loss = criterion(y, t)                # 誤差の計算
    acc  = accuracy(y,t)                  # 精度の計算
    print(f"{epoch}: loss: {loss.item()},acc:{acc}")   # 損失と精度の表示
    loss.backward()                       # 逆伝播の計算
    optimizer.step()                      # 重みの更新

0: loss: 1.4135160446166992,acc:0.32666666666666666
1: loss: 1.0877394676208496,acc:0.3377777777777778
2: loss: 0.9669662117958069,acc:0.38
3: loss: 0.7057425379753113,acc:0.8488888888888889
4: loss: 0.5306286811828613,acc:0.9555555555555556
5: loss: 0.42551159858703613,acc:0.9511111111111111
6: loss: 0.36118173599243164,acc:0.9466666666666667
7: loss: 0.2797268331050873,acc:0.9644444444444444
8: loss: 0.21629399061203003,acc:0.9644444444444444
9: loss: 0.1766132265329361,acc:0.9666666666666667
10: loss: 0.1479174792766571,acc:0.9688888888888889
11: loss: 0.12322233617305756,acc:0.9666666666666667
12: loss: 0.1026899665594101,acc:0.9755555555555555
13: loss: 0.08900816738605499,acc:0.9733333333333334
14: loss: 0.07867202162742615,acc:0.9755555555555555
15: loss: 0.07176391035318375,acc:0.9711111111111111
16: loss: 0.06649196892976761,acc:0.9733333333333334
17: loss: 0.06180734932422638,acc:0.9777777777777777
18: loss: 0.05817347764968872,acc:0.9777777777777777
19: loss: 0.0544944703578

In [9]:
# 予測データ
# テストデータを利用した精度をみる．
print("テストデータによる検証")
y_test = model(x_test)
acc = accuracy(y_test, t_test)
print("平均精度",acc)

テストデータによる検証
平均精度 1.0
